In [1]:
import pandas as pd

dataset = pd.read_csv('data_folder/dataset_raw.csv',index_col=0)
dataset.isna().sum()
dataset = dataset.dropna()
dataset

,Planet_Mass,Epsilon,Alpha,Stokes,Aspect_Ratio,SigmaSlope,Dust_gap_1,Gas_gap_1,Dust_depth_1,Gas_depth_1,Dust_gap_2,Dust_depth_2,#_DG,#_GG,Unnamed: 15
Sample#,,,,,,,,,,,,,,,
0,950,0.000128,0.0565,0.005240,0.07560,0.0870,0.541,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
1,951,0.000109,0.0352,0.001230,0.02710,0.0880,0.595,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
2,952,0.000226,0.0915,0.003910,0.00137,0.0997,0.662,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
3,953,0.000233,0.0338,0.000607,0.03160,0.0369,0.784,0.554,0.34,6.770000e-09,0.000002,0.275,6.300000e-09,2,1
4,954,0.000174,0.0850,0.004110,0.05350,0.0795,0.739,0.037,0.00,3.570000e-06,0.000000,0.000,0.000000e+00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,495,0.000209,0.0465,0.001940,0.03810,0.0749,0.822,0.788,0.00,1.420000e-07,0.000000,0.000,0.000000e+00,1,0
1176,496,0.000029,0.0238,0.006720,0.05000,0.0585,0.702,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0
1177,497,0.000092,0.0947,0.006560,0.03960,0.0384,1.100,0.697,0.00,2.310000e-11,0.000000,0.000,0.000000e+00,1,0


In [2]:
X = dataset.Planet_Mass.values
y = dataset.drop(['Planet_Mass'],axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.15, random_state = 2)
X_train.shape

(1003,)

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import IsolationForest
model = make_pipeline(StandardScaler(),GradientBoostingRegressor(random_state = 0, n_estimators =100, max_depth = 8, min_samples_split = 50,min_samples_leaf = 5, learning_rate= 0.1))
model.fit(y_train,X_train)
print(model.score(y_train, X_train))

0.6782758771711913


In [4]:
from sklearn.metrics import r2_score

X_pred = model.predict(y_val)
X_pred = X_pred.astype(int)
X_pred

array([ 584,  592,  605,  533,  497,  620,  625,  371,  650,  516,  450,
        467,  610,  482,  462,  595,  517,  472,  875,  408,  626,  622,
        644,  452,  421,  393,  483,  542,  624,  821,  689,  993,  748,
        515,  689,  552,  741,  705,  484,  555,  649,  747,  425,  724,
        494,  707,  591,  503,  379,  554,  410,  587,  687,  404,  805,
        590,  512,  685,  247,  632,  599,  527,  519,  452,  541,  836,
        544,  654,  609,  481,  609,  646,  609,  503,  809,  688,  697,
        521,  721,  507,  715,  332,  870,  310,  549,  603,  705,  688,
        735,  762,  651,  746,  534,  746,  404,  467,  681,  505,  782,
        402,  699,  860,  914,  793,  923,  621,  641,  567,  892,  620,
        678,  579,  664,  456,  729,  647,  447,  670,  666,  547,  557,
        587,  694,  457,  400,  664,  538,  598,  448,  822,  700,  836,
       1001,  376,  797,  635,  685,  509,  735,  544,  595,  475,  640,
        406,  419,  449,  670,  594,  557,  613,  5

In [5]:
df = pd.DataFrame({'Actual':X_val,'Predicted':X_pred})
df



,Actual,Predicted
0,519,584
1,570,592
2,621,605
3,933,533
4,753,497
...,...,...
172,902,474
173,548,348
174,147,624
175,1,573


In [9]:
model = IsolationForest(max_samples = 'auto',contamination = float(0.3) )
model.fit(dataset[['Planet_Mass', 'Epsilon','Alpha','Stokes','Aspect_Ratio','SigmaSlope','Dust_gap_1']])


IsolationForest(contamination=0.3)

In [11]:
dataset['score'] = model.decision_function(dataset[['Planet_Mass', 'Epsilon','Alpha','Stokes','Aspect_Ratio','SigmaSlope','Dust_gap_1']])
dataset['anomaly'] = model.predict(dataset[['Planet_Mass', 'Epsilon','Alpha','Stokes','Aspect_Ratio','SigmaSlope','Dust_gap_1']])
dataset

,Planet_Mass,Epsilon,Alpha,Stokes,Aspect_Ratio,SigmaSlope,Dust_gap_1,Gas_gap_1,Dust_depth_1,Gas_depth_1,Dust_gap_2,Dust_depth_2,#_DG,#_GG,Unnamed: 15,score,anomaly
Sample#,,,,,,,,,,,,,,,,,
0,950,0.000128,0.0565,0.005240,0.07560,0.0870,0.541,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.030008,1
1,951,0.000109,0.0352,0.001230,0.02710,0.0880,0.595,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.000140,1
2,952,0.000226,0.0915,0.003910,0.00137,0.0997,0.662,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,-0.027156,-1
3,953,0.000233,0.0338,0.000607,0.03160,0.0369,0.784,0.554,0.34,6.770000e-09,0.000002,0.275,6.300000e-09,2,1,0.015523,1
4,954,0.000174,0.0850,0.004110,0.05350,0.0795,0.739,0.037,0.00,3.570000e-06,0.000000,0.000,0.000000e+00,1,0,0.048246,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,495,0.000209,0.0465,0.001940,0.03810,0.0749,0.822,0.788,0.00,1.420000e-07,0.000000,0.000,0.000000e+00,1,0,0.068050,1
1176,496,0.000029,0.0238,0.006720,0.05000,0.0585,0.702,0.000,0.00,0.000000e+00,0.000000,0.000,0.000000e+00,0,0,0.038772,1
1177,497,0.000092,0.0947,0.006560,0.03960,0.0384,1.100,0.697,0.00,2.310000e-11,0.000000,0.000,0.000000e+00,1,0,0.033842,1


In [12]:
anomaly = dataset.loc[dataset['anomaly']==-1]
anomaly_index = list(anomaly.index)
print(anomaly_index)

[2, 7, 21, 25, 27, 28, 32, 33, 36, 39, 45, 47, 54, 55, 62, 63, 66, 69, 72, 75, 81, 82, 83, 91, 92, 95, 98, 100, 101, 104, 105, 108, 109, 110, 111, 113, 114, 115, 116, 117, 119, 120, 121, 123, 128, 136, 151, 162, 163, 166, 167, 169, 171, 172, 173, 176, 177, 180, 185, 196, 199, 204, 205, 209, 210, 212, 220, 224, 227, 236, 238, 239, 240, 244, 245, 251, 253, 255, 256, 257, 262, 263, 266, 267, 270, 271, 272, 274, 275, 276, 278, 279, 283, 284, 289, 296, 301, 302, 305, 306, 311, 320, 326, 336, 337, 338, 339, 345, 346, 347, 348, 349, 353, 355, 359, 360, 362, 364, 365, 366, 369, 370, 371, 373, 374, 376, 377, 379, 380, 381, 382, 383, 384, 387, 388, 391, 392, 398, 403, 411, 412, 415, 417, 418, 419, 421, 425, 429, 430, 431, 432, 442, 443, 446, 465, 466, 473, 490, 495, 496, 498, 503, 510, 511, 512, 514, 515, 517, 519, 520, 523, 524, 526, 530, 531, 532, 535, 539, 547, 550, 558, 565, 571, 572, 574, 577, 578, 579, 580, 583, 584, 585, 591, 599, 603, 614, 622, 632, 635, 640, 643, 645, 649, 653, 657, 661